In [ ]:
#load all source tables into pandas DataFrames
#each csv represents a logical entity required for SQL joins and BI analysis
import pandas as pd

brands = pd.read_csv("brands.csv")
influencers = pd.read_csv("influencers.csv")
campaigns = pd.read_csv("campaigns.csv")
performance = pd.read_csv("campaign_performance.csv")
payments = pd.read_csv("payments.csv")


In [ ]:
#converted all date columns to datetime format
#errors='coerce' safely converts invalid values to NaT instead of crashing
brands['onboard_date'] = pd.to_datetime(brands['onboard_date'], errors='coerce')
campaigns['campaign_start_date'] = pd.to_datetime(campaigns['campaign_start_date'], errors='coerce')
campaigns['campaign_end_date'] = pd.to_datetime(campaigns['campaign_end_date'], errors='coerce')
performance['date'] = pd.to_datetime(performance['date'], errors='coerce')
payments['payment_date'] = pd.to_datetime(payments['payment_date'], errors='coerce')

In [ ]:
#convert engagement_rate to numeric and replace missing values with median
#median is preferred to avoid skew from extreme values
#prevents broken visuals nd inconsistent grouping in BI dashboards
#engagement_rate
influencers['engagement_rate'] = pd.to_numeric(
    influencers['engagement_rate'], errors='coerce'
)
influencers['engagement_rate'] = influencers['engagement_rate'].fillna(
    influencers['engagement_rate'].median()
)
#standardize city names and replace missing/invalid entries with 'Unknown'
#city
influencers['city'] = influencers['city'].astype(str).str.title()
influencers['city'] = influencers['city'].replace({'Nan': 'Unknown'})
influencers['city'] = influencers['city'].fillna('Unknown')


In [ ]:
#fill missing campaign end dates by assuming a default 7-day duration
#prevents negative or null campaign durations which can break KPIs

campaigns['campaign_start_date'] = pd.to_datetime(
    campaigns['campaign_start_date'], errors='coerce'
)
campaigns['campaign_end_date'] = pd.to_datetime(
    campaigns['campaign_end_date'], errors='coerce'
)

campaigns['campaign_end_date'] = campaigns['campaign_end_date'].fillna(
    campaigns['campaign_start_date'] + pd.Timedelta(days=7)
)


In [ ]:
#convert all performance metrics to numeric
#missing values are set to 0 assuming no activity was recorded

numeric_cols_perf = [
    'revenue_generated', 'clicks', 'conversions',
    'impressions', 'likes', 'comments', 'shares'
]

for col in numeric_cols_perf:
    performance[col] = pd.to_numeric(performance[col], errors='coerce')
    performance[col] = performance[col].fillna(0)


In [ ]:
#standardize payment status values and handle missing entries
#missing payments are treated as 'pending'
payments['payment_status'] = payments['payment_status'].astype(str).str.capitalize()
payments['payment_status'] = payments['payment_status'].replace({'Nan': 'Pending'})
payments['payment_status'] = payments['payment_status'].fillna('Pending')

payments['amount_paid'] = pd.to_numeric(
    payments['amount_paid'], errors='coerce'
).fillna(0)


In [ ]:
#normalize text columns for consistent grouping in reports
#avoids duplicate categories like "delhi", "Delhi", "DELHI".
brands['city'] = brands['city'].astype(str).str.title()
brands['industry'] = brands['industry'].astype(str).str.capitalize()

In [ ]:
#feature engineering
#calculate campaign duration in days
#clip values to avoid zero or negative durations
#used for performance efficiency KPIs (budget per day, revenue per day)
campaigns['campaign_duration_days'] = (
    campaigns['campaign_end_date'] - campaigns['campaign_start_date']
).dt.days


campaigns['campaign_duration_days'] = campaigns['campaign_duration_days'].clip(lower=1)


In [ ]:
#identify brands running multiple campaigns
#helps analyze brand retention and loyalty

brand_campaign_count = campaigns.groupby('brand_id')['campaign_id'].transform('count')

campaigns['is_repeat_brand'] = brand_campaign_count.apply(
    lambda x: 'Yes' if x > 1 else 'No'
)


In [ ]:
#classify influencers based on median engagement rate
#helps compare high vs low performing influencers in Power BI
engagement_threshold = influencers['engagement_rate'].median()

influencers['high_engagement_flag'] = influencers['engagement_rate'].apply(
    lambda x: 'High' if x >= engagement_threshold else 'Low'
)


In [ ]:
#merge campaign budget into performance data
#enables ROI calculation at daily/performance level
perf_campaign = performance.merge(
    campaigns[['campaign_id', 'campaign_budget']],
    on='campaign_id',
    how='left'
)


In [ ]:
#create ROI flag by comparing revenue generated with campaign budget
#high ROI indicates profitable campaign performance
perf_campaign['roi_flag'] = perf_campaign.apply(
    lambda row: 'High'
    if row['revenue_generated'] > row['campaign_budget']
    else 'Low',
    axis=1
)


In [ ]:
#merge the ROI flag back into the main performance table
#keeps performance table enriched while avoiding duplication of budget data
performance = performance.merge(
    perf_campaign[['performance_id', 'roi_flag']],
    on='performance_id',
    how='left'
)


In [ ]:
#duplicate removal
#remove duplicate records from all tables
#prevents double counting issues in SQL joins and BI dashboards
brands = brands.drop_duplicates()
influencers = influencers.drop_duplicates()
campaigns = campaigns.drop_duplicates()
performance = performance.drop_duplicates()
payments = payments.drop_duplicates()


In [ ]:
#flag negative revenue values as anomalies
#helps us identify data quality issues in financial reporting
performance['revenue_anomaly'] = performance['revenue_generated'].apply(
    lambda x: 'Yes' if x < 0 else 'No'
)

In [ ]:
#identify records with no likes, comments, and shares
#helps analyze ineffective influencer campaigns
performance['zero_engagement_flag'] = performance.apply(
    lambda x: 'Yes' if (x['likes'] == 0 and x['comments'] == 0 and x['shares'] == 0) else 'No',
    axis=1
)

In [ ]:
#identify campaigns where end date is earlier than start date
#used only as a data validation check (not dropped automatically)
invalid_dates = campaigns[
    campaigns['campaign_end_date'] < campaigns['campaign_start_date']
]

invalid_dates.shape


(0, 10)

In [ ]:
#identify cases where revenue exists without impressions
#helps validate logical consistency of performance data
invalid_revenue = performance[
    (performance['impressions'] == 0) & (performance['revenue_generated'] > 0)
]

invalid_revenue.shape


(0, 13)

In [ ]:
#final duplicate verification after cleaning
print(influencers.duplicated().sum())
print(campaigns.duplicated().sum())
print(performance.duplicated().sum())


0
0
0


In [ ]:
#final schema and datatype verification before exporting cleaned data
print(brands.info())
print(influencers.info())
print(campaigns.info())
print(performance.info())
print(payments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   brand_id       100 non-null    object        
 1   brand_name     99 non-null     object        
 2   industry       100 non-null    object        
 3   city           100 non-null    object        
 4   contact_email  96 non-null     object        
 5   onboard_date   100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 4.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   influencer_id         300 non-null    object 
 1   influencer_name       300 non-null    object 
 2   platform              300 non-null    object 
 3   category              300 non-null    object 
 4   followers_count

In [ ]:
#save cleaned datset
brands.to_csv("brands_cleaned.csv", index=False)
influencers.to_csv("influencers_cleaned.csv", index=False)
campaigns.to_csv("campaigns_cleaned.csv", index=False)
performance.to_csv("performance_cleaned.csv", index=False)
payments.to_csv("payments_cleaned.csv", index=False)